In [2]:
############################################################################
#
# Romain Pujol, Lilian Welschinger
# Computational statistics, MVA
# Extension of the SAEM algorithm to left-censored data in nonlinear
# mixed-effects model: Application to HIV dynamics model
#
############################################################################

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, norm
#np.random.seed(1)

def f(t,phi):
    return np.log10(np.exp(phi[0])*np.exp(-np.exp(phi[2])*t)+np.exp(phi[1])*np.exp(-np.exp(phi[3])*t))

def viral_load(n_patient,mu=np.array([12,8,np.log(0.5),np.log(0.05)]),w=0.3*np.ones(4),sigma=0.065,table_t=np.array([1,3,7,14,28,56])):
    #returns the viral load of n_patient during the time table table_t, simulation of the data
    #also returns the real phi vector, will be useful for accuracy metrics
    t = len(table_t)
    viral_load_ = np.zeros((n_patient,t))
    phis = np.zeros((n_patient,4))
    for i in range(n_patient):
        epsilon = np.random.normal(0,np.sqrt(sigma),t)
        phi_patient = mu + np.random.multivariate_normal(np.zeros(4),np.diag(w))
        phis[i,:]=phi_patient
        for j in range(t):
            viral_load_[i,j]=f(table_t[j],phi_patient)+epsilon[j]
    return viral_load_,phis

def plot_viral_load(viral_load_,real_mu,approx_mu,table_t=np.array([1,3,7,14,28,56])):
    #takes in input the output of the viral_load function and the time table, then plots a graph representing the viral load
    shape = viral_load_.shape
    n_patient = shape[0]
    time = shape[1]

    for i in range(n_patient):
        plt.scatter(table_t,viral_load_[i,:],color='blue',marker='+',s=5)

    trace = np.log10(np.exp(real_mu[0])*np.exp(-0.5*table_t)+np.exp(real_mu[1])*np.exp(-0.05*table_t))
    plt.plot(table_t,trace,label="Real trend")
    trace_app = np.log10(np.exp(approx_mu[0])*np.exp(-np.exp(approx_mu[2])*table_t)+np.exp(approx_mu[1])*np.exp(-np.exp(approx_mu[3])*table_t))
    plt.plot(table_t,trace_app,label="Approximate trend")
    plt.xlim(0,57)
    plt.hlines(2.6,0,57,colors='red',linestyles='--',label="LOQ")
    plt.legend()
    plt.show()

    return
 
def viral_load_to_censored(viral_load_,loq=2.6):
    #input : viral_load 
    #output : censored indexes

    index = np.ones(viral_load_.shape)
    for i in range(viral_load_.shape[0]):
        for j in range(viral_load_.shape[1]):
            if viral_load_[i,j]<loq:
                index[i,j]=0

    return index
    
def p(phi,i,y,mu_current,omega_current,sigma_current,table_t=np.array([1,3,7,14,28,56])):
    #y represents the whole matrix (with imputed values of ycensored)
    #i represents the line we use
    N,K = y.shape
    #if omega_current == 0:
    #    print("error, omega = 0")
    #else:
    matrix = np.diag(1/omega_current)
    value =-(phi-mu_current).T @ matrix @ (phi-mu_current)/2
    for j in range(K):
        value -= (1/(2*sigma_current))*((y[i,j]-f(table_t[j],phi))**2)

    return np.exp(value)

def proposition_prior(current_mu,current_omega,current_phi,current_sigma,current_y):
    #current_mu, current_omega are two parameters that are being constructed in the procedure
    N,K = current_phi.shape
    update = np.zeros((N,K))
    for i in range(N):
        proposition_=np.random.multivariate_normal(current_mu,np.diag(current_omega))
        q_ratio = multivariate_normal.pdf(proposition_,current_mu,np.diag(current_omega))/multivariate_normal.pdf(current_phi[i,:],current_mu,np.diag(current_omega))
        acc_rate = np.exp(np.minimum(0,np.log(q_ratio)+np.log(p(proposition_,i,current_y,current_mu,current_omega,current_sigma))-np.log(p(current_phi[i,:],i,current_y,current_mu,current_omega,current_sigma))))
        u = np.random.rand()
        if u<acc_rate:
            update[i,:]=proposition_
        else:
            update[i,:]=current_phi[i,:]
    return update

def proposition_multidim_rw(current_phi,current_omega,current_sigma,current_y,current_mu,lambd=1):
    #multidimensional random walk proposal
    N,K = current_phi.shape
    proposition = np.zeros((N,K))
    for i in range(N):
        proposition_=np.random.multivariate_normal(current_phi[i,:],lambd*np.diag(current_omega))
        acc_rate = np.exp(np.minimum(0,np.log(p(proposition_,i,current_y,current_mu,current_omega,current_sigma))-np.log(p(current_phi[i,:],i,current_y,current_mu,current_omega,current_sigma))))
        u = np.random.rand()
        if u<acc_rate:
            proposition[i,:]=proposition_
        else:
            proposition[i,:]=current_phi[i,:]
    return proposition

def proposition_unidim_rw(current_phi,lambd,current_y,current_mu,current_omega,current_sigma):
    #unidimensional random walk proposal
    N,K = current_phi.shape
    proposition = np.copy(current_phi)
    for i in range(N):
        line_i = np.copy(current_phi[i,:])
        for j in range(K):
            step = np.random.normal(0,lambd)
            proposition[i,j]+=step
            acc_rate = np.minimum(1,p(proposition[i,:],i,current_y,current_mu,current_omega,current_sigma)/p(line_i,i,current_y,current_mu,current_omega,current_sigma))
            u = np.random.rand()
            if u>acc_rate:
                proposition[i,j]=line_i[j]
    
    return proposition


def hm_algorithm_update(current_y,current_phi,current_omega,current_mu,current_sigma,lambd,method):
    if method==1:
        update= proposition_prior(current_mu,current_omega,current_phi,current_sigma,current_y)
    elif method==2:
        update=proposition_multidim_rw(current_phi,current_omega,current_sigma,current_y,current_mu,lambd=1)
    elif method==3:
        update=proposition_unidim_rw(current_phi,lambd,current_y,current_mu,current_omega,current_sigma)
    return update    
    


def update_y_cens(phi,t,sigma,LOQ):
    m=f(t,phi) 
    C=(f(t,phi)-LOQ)/sigma
    alpha = (C+np.sqrt(C**2+4))/2
    u = np.random.rand()
    x = (-1/alpha)*np.log(1-u)+C
    proba = np.exp(-(x-alpha)**2/2)
    u = np.random.rand()
    while u > proba :
        u = np.random.rand()
        x = -1/alpha * np.log(1 - u) + C
        proba = np.exp(-(x-alpha)**2/2)
        u = np.random.rand()
    y = m - sigma*x
    return y


def S(y,phi,time_t=np.array([1,3,7,14,28,56])):
    N,K = y.shape
    S_1 = np.sum(phi,axis=0)
    S_2 = np.sum(phi**2,axis=0)
    S_3=0
    for i in range(N):
        for j in range(K):
            S_3+=((y[i,j]-f(time_t[j],phi[i,:]))**2)
        
    return S_1,S_2,S_3

def update_s(s_m,gamma,y,phi):
    s_1,s_2,s_3 = S(y,phi)
    if gamma == 1:
        return s_1,s_2,s_3
    else:
        return s_m[0] +gamma*(s_1-s_m[0]),s_m[1] +gamma*(s_2-s_m[1]),s_m[2] +gamma*(s_3-s_m[2])


def update_mu_w_sigma(S_1,S_2,S_3,N,K):
    new_mu = S_1/N
    new_w = S_2/N- (new_mu)**2
    new_sigma = S_3/(N*K)
    return new_mu,new_w,new_sigma

def SAEM(omega_0,mu_0,sigma_0,y_0,phi_0,s_0,iteration,method,lambd=1,t=np.array([1,3,7,14,28,56]),LOQ=2.6,M1 = 1000):

    y = np.copy(y_0)
    phi = np.copy(phi_0)
    omega = np.copy(omega_0)
    mu = np.copy(mu_0)
    sigma = np.copy(sigma_0)
    s = s_0
    N,K = y.shape
    mu_evolve = [mu]
    omega_evolve = [omega]
    sigma_evolve = [sigma]

    index_censored = np.where(y <= LOQ)

    for i in range(iteration):
        #########################################
        # S step : simulation of the missing data
        #########################################

        #simulate phi
        phi = hm_algorithm_update(y,phi,omega,mu,sigma,lambd,method)
        #simulate y_censored
        for k,j in zip(index_censored[0],index_censored[1]):
            y[k,j] = update_y_cens(phi[k,:],t[j],sigma,LOQ)

        #########################################
        # SA step 
        #########################################

        #update s
        if i <= M1:
            gamma = 1
        else:
            gamma = 1/(i-M1)
        
        s = update_s(s,gamma,y,phi)

        #########################################
        # M step 
        #########################################

        mu,omega,sigma = update_mu_w_sigma(s[0],s[1],s[2],N,K)
        mu_evolve.append(mu)
        sigma_evolve.append(sigma)
        omega_evolve.append(omega)

    return mu,omega,sigma,y,mu_evolve,sigma_evolve,omega_evolve





################################################################
# Real parameters and data
################################################################

mu_=np.array([12,8,np.log(0.5),np.log(0.05)])
w=0.3*np.ones(4)
sigma=0.065
table_t=np.array([1,3,7,14,28,56])
y,phi = viral_load(100)


################################################################
# Input of SAEM, censored data
################################################################

starting_y=np.copy(y)
N,K = starting_y.shape
for i in range(N):
    for j in range(K):
        if starting_y[i,j]<2.6:
            ##############################################################
            # ici on peut expérimetner avec la valeur à mettre au départ
            #par exemple 2.6, 1.3, 0
            #ou d'autres idées un peu plus intéressante en calculant
            #l'écart avec 2.6 avec la valeur précédente,
            #on peut peut être imputer une valeur intelligente :
            # si la valeur d'avant est 2.7, on peut se douter que celle
            # d'après sera vraisemblablement plus faible que si la valeur
            #précédente était 3.6
            ##############################################################
            starting_y[i,j]=2.6 



#######################################################################
# ici on peut aussi essayer de donner des valeurs intéressantes
# pour certaines variables, j'y ai pas pensé encore
#j'ai juste mis des trucs aléatoires proche de la vraie valeur
#######################################################################

phi_test = phi + 2*(np.random.rand(4))-1
s_0 = (np.zeros(4), np.zeros(4), 0)
starting_mu=mu_+4*(np.random.rand(4))-2
starting_w= w +0.2*(np.random.rand(4))-0.1
starting_sigma = 0.4


#problème avec la méthode 1 dans la partie d'exploration : quand on est en dessous de M1 parce qu'on passe sur une matrice
#qui n'est plus SDP en faisant varier les valeurs de theta (donc les valeurs de omega), 
#en revanche, les méthodes 2 et 3 fonctionnent plutôt bien, je n'ai pas vu de problème

method = 2
"""
À la ligne suivante tu mets y en sortie de SAEM, mais ne faudrait-il pas mettre un y_est parce que t'as
déjà une variable y qui correspond aux vraies datas ? 
"""
#mu,omega,sigma,y,mu_evolve,sigma_evolve,omega_evolve=SAEM(starting_w,starting_mu,starting_sigma,starting_y,phi_test,s_0,1500,method,M1=500)

#alors oui des valeurs ne convergent pas, notamment celles de w mais dans l'article c'est pareil
#et les mecs de l'année précédente aussi (et ils convergent moins bien pour les autres valeurs)

"""

Montre les paramètres et la tendance
plot_viral_load(y,mu_,mu)

"""

"""
Graphique des paramèters



mu_evolve=np.array(mu_evolve)
omega_evolve = np.array(omega_evolve)
sigma_evolve = np.array(sigma_evolve)

fig, axs = plt.subplots(3, 4, figsize=(10, 8))

axs[0, 0].plot(mu_evolve[:,0], label='ln(P1)', color='blue')
axs[0, 0].axhline(12, color='r', linestyle='--')  
axs[0, 0].set_ylim(10,14)
axs[0, 0].legend()

axs[0, 1].plot(mu_evolve[:,1], label='ln(P2)', color='blue')
axs[0, 1].axhline(8, color='r', linestyle='--')  
axs[0, 1].set_ylim(6,10)
axs[0, 1].legend()

axs[0, 2].plot(mu_evolve[:,2], label='ln(lambda1)', color='blue')
axs[0, 2].axhline(np.log(0.5), color='r', linestyle='--')  
axs[0, 2].set_ylim(np.log(0.5)-2,np.log(0.5)+2)
axs[0, 2].legend()

axs[0, 3].plot(mu_evolve[:,3], label='ln(lambda2)', color='blue')
axs[0, 3].axhline(np.log(0.05), color='r', linestyle='--')  
axs[0, 3].set_ylim(np.log(0.05)-2,np.log(0.05)+2)
axs[0, 3].legend()

axs[1, 0].plot(omega_evolve[:,0], label='w_1', color='blue')
axs[1, 0].axhline(0.3, color='r', linestyle='--')  
axs[1, 0].set_ylim(0,1)  
axs[1, 0].legend()

axs[1, 1].plot(omega_evolve[:,1], label='w_2', color='blue')
axs[1, 1].axhline(0.3, color='r', linestyle='--')  
axs[1, 1].set_ylim(0,1)  
axs[1, 1].legend()

axs[1, 2].plot(omega_evolve[:,2], label='w_3', color='blue')
axs[1, 2].axhline(0.3, color='r', linestyle='--')
axs[1, 2].set_ylim(0,1)
axs[1, 2].legend()

axs[1, 3].plot(omega_evolve[:,3], label='w_4', color='blue')
axs[1, 3].axhline(0.3, color='r', linestyle='--')
axs[1, 3].set_ylim(0,1)  
axs[1, 3].legend()

axs[2, 0].plot(sigma_evolve, label='sigma', color='blue')
axs[2, 0].axhline(0.065, color='r', linestyle='--')
axs[1, 0].set_ylim(0,0.5)  
axs[2, 0].legend()

axs[2, 1].axis('off')
axs[2, 2].axis('off')
axs[2, 3].axis('off')

plt.tight_layout()
plt.show()
"""


"""
Relative bias and relative root mean square error (RMSE) of parameters for the SAEM method. This being taken
on nb_trials trials. 


nb_trials = 15
iterations = 500
method = 2
SAEM_params = [starting_w, starting_mu, starting_sigma, starting_y, phi_test, s_0, iterations, method]
real_params = [mu_, w, sigma]

def plot_bias_RMSE(nb_trials, SAEM_params, real_params):
    
    mu_estimates = []
    omega_estimates = []
    sigma_estimates = []
    
    starting_w, starting_mu, starting_sigma, starting_y, phi_test, s_0, iterations, method = SAEM_params
    mu_, w, sigma = real_params
    
    for _ in range(nb_trials):
        mu_est, omega_est, sigma_est, _, _, _, _ = SAEM(starting_w, starting_mu, starting_sigma, starting_y, 
                                                        phi_test, s_0, iterations, method, M1=500)

        mu_estimates.append(mu_est)
        omega_estimates.append(omega_est)
        sigma_estimates.append(sigma_est)
        
    mu_estimates = np.array(mu_estimates)
    omega_estimates = np.array(omega_estimates)
    sigma_estimates = np.array(sigma_estimates)

    # Compute the bias and RMSE of each parameters
    mu_bias = np.mean(mu_estimates - mu_, axis=0)
    mu_rmse = np.sqrt(np.mean((mu_estimates - mu_)**2, axis=0))

    omega_bias = np.mean(omega_estimates - w, axis=0)
    omega_rmse = np.sqrt(np.mean((omega_estimates - w)**2, axis=0))

    sigma_bias = np.mean(sigma_estimates - sigma)
    sigma_rmse = np.sqrt(np.mean((sigma_estimates - sigma)**2))

    print("Parameters \t Relative Bias \t\t Relative RMSE")
    for i, param in enumerate(["ln(P1)", "ln(P2)", "ln(𝜆1)", "ln(𝜆2)"]):
        print(f"  {param}\t     {np.abs(mu_bias[i]/mu_[i]):.2f}\t\t      {np.abs(mu_rmse[i]/mu_[i]):.2f}")
    for i, param in enumerate(["omega1", "omega2", "omega3", "omega4"]):
        print(f"  {param}\t     {np.abs(omega_bias[i]/w[i]):.2f}\t\t      {np.abs(omega_rmse[i]/w[i]):.2f}")
    print(f"  sigma\t   \t     {np.abs(sigma_bias/sigma):.2f}\t\t      {np.abs(sigma_rmse/sigma):.2f}")
    
    return

plot_bias_RMSE(nb_trials, SAEM_params, real_params)
 
"""

""" Je t'ai mis ce que donne le plot quand nb_trials = 15 et iterations = 500. Le programme prend 
son petit temps à tourner. 

Parameters   Relative Bias     Relative RMSE
  ln(P1)          0.02              0.02
  ln(P2)          0.02              0.02
  ln(𝜆1)          0.02              0.08
  ln(𝜆2)          0.11              0.11
  omega1          0.54              0.67
  omega2          0.45              0.55
  omega3          0.06              0.14
  omega4          0.88              0.89
  sigma           0.20              0.21
  
"""










#travailler sur l'acceptance rate de proposition_multidim_rw et proposition_unidim_rw avec le lambda
#sur la première imputation

" Je t'ai mis ce que donne le plot quand nb_trials = 15 et iterations = 500. Le programme prend \nson petit temps à tourner. \n\nParameters   Relative Bias     Relative RMSE\n  ln(P1)          0.02              0.02\n  ln(P2)          0.02              0.02\n  ln(𝜆1)          0.02              0.08\n  ln(𝜆2)          0.11              0.11\n  omega1          0.54              0.67\n  omega2          0.45              0.55\n  omega3          0.06              0.14\n  omega4          0.88              0.89\n  sigma           0.20              0.21\n  \n"

$$
\text{Relative bias} = \frac{\sum_{i=1}^N x_i^{estimated}-x_{true}}{Nx_{true}}
$$

$$
\text{Relative RMSE} = \frac{\sqrt{\frac{\sum_{i=1}^N (x_i^{estimated}-x_{true})^2}{N}}}{x_{true}}
$$

The NLMEM (Nonlinear mixed effects model) is defined as follows : 

\begin{equation*}
\begin{cases}
y_{ij} &= f(\phi_i, t_{ij}) + g\left(\phi_i, t_{ij}\right)\epsilon_{ij} \\
\epsilon_{ij} &\sim \mathcal{N}(0, \sigma^2 I_{n_i}) \\
\phi_i &= X_i\mu + b_i, \text{ with }b_i \sim \mathcal{N}(0, \Omega)
\end{cases}
\end{equation*}

The parameters $\theta$ of the models are : $\theta = (\mu,\sigma^2,\Omega)$.

First, we will compute the complete likelihood of the $i$-th subject :

$
\begin{aligned}
\textit{L}\left(y_i^{obs},y_i^{cens},\phi_i;\theta\right) &= \log\left(p\left(y_i^{obs}, y_i^{cens},\phi_i;\theta\right)\right) \\
&= \sum_{(i,j)\in I_{obs}} \left(\log\left(p\left(y_{ij}^{obs}|\phi_i;\theta\right)\right)\right) + \sum_{(i,j)\in I_{cens}} \left(\log\left(p\left(y_{ij}^{cens}|\phi_i;\theta\right)\right)\right)+ \log\left(p\left(\phi_i;\theta\right)\right)
\end{aligned}
$

where : 
$
\begin{aligned}
p\left(y_{ij}^{obs}|\phi_i;\theta\right) &= \pi\left(y_{ij};f\left(\phi_i,t_{ij}\right),\sigma^2g^2\left(\phi_i,t_{ij}\right)\right)\mathbb{1}_{\{y_{ij} \geq LOQ\}}, \text{ if } (i,j)\in I_{obs}\\
p\left(y_{ij}^{cens}|\phi_i;\theta\right) &= \pi\left(y_{ij};f\left(\phi_i,t_{ij}\right),\sigma^2g^2\left(\phi_i,t_{ij}\right)\right)\mathbb{1}_{\{y_{ij} \leq LOQ\}}, \text{ if } (i,j)\in I_{cens}
\end{aligned}
$ 
and $\pi(x; m, v)$ is the probability density function of the Gaussian distribution with mean $m$ and variance $v$, evaluated at $x$.

According to the NLMEM model, we have : 
$$
\forall i = 1, \dots, N, \;p\left(\phi_i;\theta\right) = \frac{1}{\sqrt{(2\pi)^p\det\left(\Omega\right)}} \exp\left(-\frac{1}{2}\left(\phi_i - X_i\mu\right)^T\Omega^{-1}\left(\phi_i - X_i\mu\right)\right)
$$

Then, we deduce that the complete likelihood of the $i$-th subject is equal to : 

$
\begin{aligned}
\textit{L}\left(y_i^{obs},y_i^{cens},\phi_i;\theta\right) &= -\sum_{j=1}^{n_i} \frac{\left(y_{ij} - f\left(\phi_i, t_{ij}\right)\right)^2}{2\sigma^2 g^2\left(\phi_i, t_{ij}\right)} - \frac{n_i}{2}\left(\log(2\pi)+\log\left(\sigma^2\right) \right) - \sum_{j=1}^{n_i} \log\left(g\left(\phi_i, t_{ij}\right)\right) \\ &- \frac{1}{2}\left(p\log(2\pi)+ \log\det\left(\Omega\right)\right) -\frac{1}{2}\left(\phi_i - X_i\mu\right)^T\Omega^{-1}\left(\phi_i - X_i\mu\right)
\end{aligned}
$

And finaly, the complete likelihood of the model is equal to : 

$
\begin{aligned}
\textit{L}\left(y,\phi;\theta\right) &= -\sum_{i=1}^N \sum_{j=1}^{n_i} \frac{\left(y_{ij} - f\left(\phi_i, t_{ij}\right)\right)^2}{2\sigma^2 g^2\left(\phi_i, t_{ij}\right)} - \frac{k}{2}\left(\log(2\pi)+\log\left(\sigma^2\right) \right) - \sum_{i=1}^N \sum_{j=1}^{n_i} \log\left(g\left(\phi_i, t_{ij}\right)\right) \\ &- \frac{N}{2}\left(p\log(2\pi)+ \log\det\left(\Omega\right)\right) -\frac{1}{2}\sum_{i=1}^N\left(\phi_i - X_i\mu\right)^T\Omega^{-1}\left(\phi_i - X_i\mu\right)
\end{aligned}
$

where : 
$$
k = \sum_{i=1}^N n_i
$$

In the following, for the application, $\Omega$ will be taken diagonally as $\Omega = \text{diag}(\omega_1^2,\omega_2^2,\omega_3^2,\omega_4^2)$, $g=1$ and $X_i$ independant of $i$ so we will write $X$.
$
\begin{aligned}
\textit{L}\left(y,\phi;\theta\right) &= -\sum_{i=1}^N \sum_{j=1}^{n_i} \frac{\left(y_{ij} - f\left(\phi_i, t_{ij}\right)\right)^2}{2\sigma^2} - \frac{k}{2}\left(\log(2\pi)+\log\left(\sigma^2\right) \right) \\ &- \frac{N}{2}\left(p\log(2\pi)+ \log\det\left(\Omega\right)\right) -\frac{1}{2}\sum_{i=1}^N\left(\tilde{\phi}_i^T\tilde{\omega} - \phi_i^T\Omega^{-1}X\mu +(X\mu)^T\Omega^{-1}X\mu\right)
\end{aligned}
$

where we denote : 
$$
\tilde{\phi}_i = \begin{pmatrix}\phi_{1i}^2\\\dots\\\phi_{4i}^2\end{pmatrix} \text{ and } \tilde{\omega} = \begin{pmatrix}\omega_1^{-2}\\\dots\\\omega_4^{-2}\end{pmatrix}
$$

Finaly, we can write the complete likelihood of the model in the following form : 

$$
\textit{L}\left(y,\phi;\theta\right) = -\Lambda(\theta) + \langle S(y,\phi),\Phi(\theta) \rangle
$$

where : 
\begin{equation*}
\begin{cases}
\Lambda(\theta) &= \frac{k}{2}\left(\log(2\pi)+\log\left(\sigma^2\right)\right) + \frac{N}{2}\left(p\log(2\pi)+ \log\det\left(\Omega\right)\right) + \frac{1}{2}\sum_{i=1}^N (X\mu)^T\Omega^{-1}X\mu\\
S(y,\phi) &= \left(\sum_{i=1}^N \phi_i, \sum_{i=1}^N \tilde{\phi}_i, \sum_{i=1}^N \sum_{j=1}^{n_i}\left(y_{ij} - f\left(\phi_i, t_{ij}\right)\right)^2 \right)^T \\
\Phi(\theta) &= \left(\frac{1}{2}\Omega^{-1}X\mu, -\frac{1}{2}\tilde{\omega}, -\frac{1}{2\sigma^2}\right)^T
\end{cases}
\end{equation*}

Then, our complete likelihood belongs to the regular curved exponential family.

Here, we have : 
$$
f(\phi_i,t_{ij}) = \log_{10}\left(P_{1i}\text{e}^{-\lambda_{1i}t_{ij}}+P_{2i}\text{e}^{-\lambda_{2i}t_{ij}}\right)
$$

# Imputation initiale

What we can do is compute the better fitting curve to the observed data to initiate $\phi_i$ for each patient. With the parameters we will find, we can thus impute the first censored values and it will give the first guess $y_{censored}$.

In [6]:
import numpy as np
from scipy.optimize import curve_fit

t=np.array([1,3,7,14,28,56])
y_i,phi = viral_load(1)
y_i = y_i[0]

def model(t, P1, lambda1, P2, lambda2):
    return np.log10(np.exp(P1) * np.exp(-lambda1 * t) + np.exp(P2) * np.exp(-lambda2 * t))

print(y_i)
initial_guess = [100, 1, 10, 0.05]  
popt, pcov = curve_fit(model, t, y_i, p0=initial_guess)

print("Real values : P1 =", phi[0][0], "log(lambda1) =", phi[0][2], "P2 =", phi[0][1], "log(lambda2) =", phi[0][3])

P1_opt, lambda1_opt, P2_opt, lambda2_opt = popt
print("P1 =", P1_opt, "log(lambda1) =", np.log(lambda1_opt), "P2 =", P2_opt, "log(lambda2) =", np.log(lambda2_opt))

y_fit = model(t[-1], P1_opt, lambda1_opt, P2_opt, lambda2_opt)
print(y_fit)



[5.14289138 4.45817227 3.42778787 2.52243023 2.03999872 0.83023301]
Real values : P1 = 12.4025407585337 log(lambda1) = -0.4822987721909587 P2 = 7.285360082437336 log(lambda2) = -2.4231198278389883
P1 = 12.471576253446617 log(lambda1) = -0.34490872839448694 P2 = 7.185951910179049 log(lambda2) = -2.370249288610988
0.8478887872604784


We can compare the different imputations.

In [4]:
np.random.seed(2)

total_0 = 0
total_mean = 0
total_2 = 0
total_fit = 0

t = np.array([i for i in range(56)])

iter = 200

for _ in range(iter):

    y, phi = viral_load(2, table_t=t)

    imput_2 = np.copy(y)
    imput_mean = np.copy(y)
    imput_zero = np.copy(y)
    imput_fit = np.copy(y)
    N, K = imput_2.shape

    error_0 = 0
    error_mean = 0
    error_2 = 0
    error_fit = 0
    number = 0

    for i in range(N):
        for j in range(K):
            if y[i, j] <= 2.6:
                number += 1
                imput_2[i, j] = 2.6
                error_2 += (y[i, j] - 2.6) ** 2
                imput_mean[i, j] = 1.3
                error_mean += (y[i, j] - 1.3) ** 2
                imput_zero[i, j] = 0
                error_0 += (y[i, j]) ** 2

    for i in range(N):
        censored_idx = np.where(y[i] <= 2.6)[0]  
        initial_guess = [100, 1, 10, 0.05]
        time_observed = [t[k] for k in range(26) if k not in censored_idx]
        index_observed = [k for k in range(26) if k not in censored_idx]
        
        if len(time_observed) > 1: 
            popt, pcov = curve_fit(model, time_observed, y[i][index_observed], p0=initial_guess)
            P1_opt, lambda1_opt, P2_opt, lambda2_opt = popt
            for k in range(len(censored_idx)):  
                imput_fit[i, censored_idx[k]] = model(t[censored_idx[k]], P1_opt, lambda1_opt, P2_opt, lambda2_opt)
                if imput_fit[i, censored_idx[k]] >= 2.6:
                    imput_fit[i, censored_idx[k]] = 2.6
                elif imput_fit[i, censored_idx[k]] < 0:
                    imput_fit[i, censored_idx[k]] = 0

                error_fit += (y[i, censored_idx[k]] - imput_fit[i, censored_idx[k]]) ** 2

    total_0 += error_0 / number
    total_mean += error_mean / number
    total_2 += error_2 / number
    total_fit += error_fit / number

print(f'Error with 0, {total_0 / iter}')
print(f'Error with 1.3, {total_mean / iter}')
print(f'Error with 2.6, {total_2 / iter}')
print(f'Error with fit, {total_fit / iter}')


C:\Users\romai\AppData\Local\Temp\ipykernel_22868\3868173010.py:46: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(model, time_observed, y[i][index_observed], p0=initial_guess)
C:\Users\romai\AppData\Local\Temp\ipykernel_22868\73455518.py:9: RuntimeWarning: overflow encountered in exp
  return np.log10(np.exp(P1) * np.exp(-lambda1 * t) + np.exp(P2) * np.exp(-lambda2 * t))


Error with 0, 4.640201902397899
Error with 1.3, 0.8938375003131271
Error with 2.6, 0.5274730982283551
Error with fit, 0.41675873635462457
